In [8]:
from azureml.core import Workspace, Dataset

subscription_id = '230da710-78c2-4cc1-9e21-36bccff41cae'
resource_group = 'RESEARCH-DEVELOPMENT'
workspace_name = 'aml-development'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, "volve_clean_dataset")
dataset.download(target_path='.', overwrite=False)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/visualizationdmim-cpu/code/Users/Fuad.Maulana/clean_data.csv']

In [37]:
df = pd.read_csv("clean_data.csv")
test_df = df.iloc[len(df)-732:]
test_df.drop(["BORE_GAS_VOL", "BORE_WAT_VOL"], axis=1, inplace=True)
test_df.to_csv("test_df.csv")

/tmp/ipykernel_8155/2943831687.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(["BORE_GAS_VOL", "BORE_WAT_VOL"], axis=1, inplace=True)


In [38]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 2484 to 3215
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DATEPRD                   732 non-null    object 
 1   AVG_DOWNHOLE_PRESSURE     732 non-null    float64
 2   AVG_DOWNHOLE_TEMPERATURE  732 non-null    float64
 3   AVG_DP_TUBING             732 non-null    float64
 4   AVG_WHP_P                 732 non-null    float64
 5   AVG_WHT_P                 732 non-null    float64
 6   DP_CHOKE_SIZE             732 non-null    float64
 7   BORE_OIL_VOL              732 non-null    float64
dtypes: float64(7), object(1)
memory usage: 45.9+ KB


In [39]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [(f'{df.columns[j]}(t-{i})') for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [(f'{df.columns[j]}(t)') for j in range(n_vars)]
        else:
            names += [(f'{df.columns[j]}(t+{i})') for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# frame as supervised learning
test_df.set_index("DATEPRD", inplace=True)
n_in, n_out = 2, 1
col_n_in = len(test_df.columns)*n_in
reframed = series_to_supervised(test_df, n_in, n_out)

# drop columns we don't want to predict
reframed.drop(reframed.columns[list(range(col_n_in, col_n_in+len(test_df.columns)-1))], axis=1, inplace=True)
reframed

,AVG_DOWNHOLE_PRESSURE(t-2),AVG_DOWNHOLE_TEMPERATURE(t-2),AVG_DP_TUBING(t-2),AVG_WHP_P(t-2),AVG_WHT_P(t-2),DP_CHOKE_SIZE(t-2),BORE_OIL_VOL(t-2),AVG_DOWNHOLE_PRESSURE(t-1),AVG_DOWNHOLE_TEMPERATURE(t-1),AVG_DP_TUBING(t-1),AVG_WHP_P(t-1),AVG_WHT_P(t-1),DP_CHOKE_SIZE(t-1),BORE_OIL_VOL(t-1),BORE_OIL_VOL(t)
DATEPRD,,,,,,,,,,,,,,,
2014-12-03,213.025456,82.530965,198.160056,67.230199,31.872970,30.528663,448.71,214.756528,81.923593,194.452604,73.458704,31.817019,35.182287,460.07,456.05
2014-12-04,214.756528,81.923593,194.452604,73.458704,31.817019,35.182287,460.07,215.829434,81.738377,192.702049,76.898942,32.995571,37.648619,456.05,662.11
2014-12-05,215.829434,81.738377,192.702049,76.898942,32.995571,37.648619,456.05,216.136133,81.964996,189.963722,80.453335,33.880065,39.287733,662.11,905.35
2014-12-06,216.136133,81.964996,189.963722,80.453335,33.880065,39.287733,662.11,216.209973,82.934040,186.712924,83.903466,41.576689,38.846555,905.35,605.76
2014-12-07,216.209973,82.934040,186.712924,83.903466,41.576689,38.846555,905.35,217.281907,81.990201,201.825268,69.917991,32.540850,32.199197,605.76,532.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-11-27,149.863207,48.828456,145.716024,7.683192,17.051464,0.000000,0.00,149.863207,48.828456,145.716024,7.683192,17.051464,0.000000,0.00,0.00
2016-11-28,149.863207,48.828456,145.716024,7.683192,17.051464,0.000000,0.00,149.863207,48.828456,145.716024,7.683192,17.051464,0.000000,0.00,0.00
2016-11-29,149.863207,48.828456,145.716024,7.683192,17.051464,0.000000,0.00,149.863207,48.828456,145.716024,7.683192,17.051464,0.000000,0.00,0.00


In [40]:
# Scaling Oil Data
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
scaler_oil = MinMaxScaler()
oil_scaled = scaler_oil.fit_transform(reframed) 
oil_scaled_df = pd.DataFrame(oil_scaled, index=reframed.index, columns=reframed.columns)
oil_scaled_df

,AVG_DOWNHOLE_PRESSURE(t-2),AVG_DOWNHOLE_TEMPERATURE(t-2),AVG_DP_TUBING(t-2),AVG_WHP_P(t-2),AVG_WHT_P(t-2),DP_CHOKE_SIZE(t-2),BORE_OIL_VOL(t-2),AVG_DOWNHOLE_PRESSURE(t-1),AVG_DOWNHOLE_TEMPERATURE(t-1),AVG_DP_TUBING(t-1),AVG_WHP_P(t-1),AVG_WHT_P(t-1),DP_CHOKE_SIZE(t-1),BORE_OIL_VOL(t-1),BORE_OIL_VOL(t)
DATEPRD,,,,,,,,,,,,,,,
2014-12-03,0.763118,0.954591,0.615289,0.684897,0.288685,0.603525,0.119907,0.783780,0.937441,0.572867,0.748659,0.287767,0.695523,0.122943,0.121868
2014-12-04,0.783780,0.937441,0.572867,0.748659,0.287767,0.695523,0.122943,0.796587,0.932211,0.552836,0.783877,0.307112,0.744281,0.121868,0.176933
2014-12-05,0.796587,0.932211,0.552836,0.783877,0.307112,0.744281,0.121868,0.800248,0.938610,0.521503,0.820264,0.321631,0.776684,0.176933,0.241933
2014-12-06,0.800248,0.938610,0.521503,0.820264,0.321631,0.776684,0.176933,0.801129,0.965973,0.484307,0.855584,0.447969,0.767963,0.241933,0.161875
2014-12-07,0.801129,0.965973,0.484307,0.855584,0.447969,0.767963,0.241933,0.813924,0.939322,0.657227,0.712412,0.299648,0.636550,0.161875,0.142341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-11-27,0.009194,0.002920,0.015206,0.075304,0.045394,0.000000,0.000000,0.009194,0.002920,0.015206,0.075304,0.045394,0.000000,0.000000,0.000000
2016-11-28,0.009194,0.002920,0.015206,0.075304,0.045394,0.000000,0.000000,0.009194,0.002920,0.015206,0.075304,0.045394,0.000000,0.000000,0.000000
2016-11-29,0.009194,0.002920,0.015206,0.075304,0.045394,0.000000,0.000000,0.009194,0.002920,0.015206,0.075304,0.045394,0.000000,0.000000,0.000000


In [41]:
oil_scaled_df.to_csv("test_set.csv", index=False)

In [49]:
test_json = oil_scaled_df.drop("BORE_OIL_VOL(t)", axis=1)
test = {"data": test_json.values.tolist()}
# print(type(test))
# print(test)
import json
with open("test.json", "w") as fp:
    json.dump(test, fp)